# Chapter 15. More About Type Hints
10:33 -
 - Overloaded function signatures
 - `typing.TypedDict` for type hinting `dicts` used as records
 - Type Casting
 - Runtime access to type hints
 - generic types
  - declaring a generic class
  - variance: invariant, convariant, contravariant
  - generic static protocols

## Overloaded Signatures

In [1]:
help(sum)

Help on built-in function sum in module builtins:

sum(iterable, /, start=0)
    Return the sum of a 'start' value (default: 0) plus an iterable of numbers
    
    When the iterable is empty, return the start value.
    This function is intended specifically for use with numeric values and may
    reject non-numeric types.



In [2]:
# mysum.py
import functools
import operator
from collections.abc import Iterable
from typing import overload, Union, TypeVar

T = TypeVar('T')
S = TypeVar('S')

@overload
def sum(it: Iterable[T]) -> Union[T, int]: ...

@overload
def sum(it: Iterable[T], /, start: S) -> Union[T, S]: ...
def sum(it, /, start=0):
  return functools.reduce(operator.add, it, start)

In [4]:
sum([1, 2, 3])

6

In [9]:
max([1, 2, 3])
max(1, 2, 3)
max([])

ValueError: max() arg is an empty sequence

In [19]:
# mymax.py
from collections.abc import Callable, Iterable
from typing import Protocol, Any, TypeVar, overload, Union

class SupportsLessThan(Protocol):
  def __lt__(self, other: Any) -> bool: ...

T = TypeVar('T')
LT = TypeVar('LT', bound=SupportsLessThan)
DT = TypeVar('DT')

MISSING = object()  # unique object instance used as sentinel
EMPTY_MSG = 'max() arg is an empty sequence'

@overload
def max(__arg1: LT, __arg2: LT, *args: LT, key: None = ...) -> LT:
  ...

@overload
def max(__arg1: T, __arg2: T, *args: T, key: Callable[[T], LT] = ...) -> T:
  ...

@overload
def max(__iterable: Iterable[LT], *, key: None = ...) -> LT:
  ...

@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT] = ...) -> T:
  ...

@overload
def max(__iterable: Iterable[LT], *, key: None = ...,
        default: DT) -> Union[LT, DT]:
  ...

@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT] = ...,
        default: DT) -> Union[T, DT]:
  ...

def max(first, *args, key=None, default=MISSING):
  if args: # if there is a second positional argument
    series = args
    candidate = first
  else:
    series = iter(first)
    try:
      candidate = next(series)
    except StopIteration:
      if default is not MISSING:
        return default
      raise ValueError(EMPTY_MSG) from None
  if key is None:
    for current in series:
      if candidate < current:
        candidate = current
  else:
    candidate_key = key(candidate)
    for current in series:
      current_key = key(current)
      if candidate_key < current_key:
        candidate = current
        candidate_key = current_key
  return candidate

In [15]:
def test(a, *, b, c=3):
  print(a, b, c)

In [18]:
test(3, b=2)

3 2 3


In [14]:
test(a=3, b=2)

TypeError: test() got some positional-only arguments passed as keyword arguments: 'a'

## TypedDict

 - It's tempting to use `TypedDict` to protect against errors while handling dynamic data structures like JSON API responses. But the example here make clear that correct handling of JSON must be done at runtime.

In [20]:
from typing import TypedDict

class BookDict(TypedDict):
  isbn: str
  title: str
  authors: list[str]
  pagecount: int

At first glance, `typing.TypedDict` may seem like a data class builder, similar to `typing.NamedDict`.

But, it only exists for the benefit of type checkers! (No runtime effect)

In [21]:
pp = BookDict(title="Programming Pearls",
              authors='Jon Bentley', # no type checking in runtime
              isbn='0201657880',
              pagecount=256)

In [22]:
pp

{'title': 'Programming Pearls',
 'authors': 'Jon Bentley',
 'isbn': '0201657880',
 'pagecount': 256}

In [23]:
type(pp) # result of calling BookDict is a plain dict

dict

In [25]:
BookDict.__annotations__

{'isbn': str, 'title': str, 'authors': list[str], 'pagecount': int}

In [34]:
# demo_books.py
from typing import TYPE_CHECKING

def demo() -> None: # Don't forget to add None so that Mypy doesn't ignore the fcn
  book = BookDict(
      isbn='0134757599',
      authors=['Martin Fowler', 'Kent Beck'],
      title='Refactoring, 2e',
      pagesount=478
  )
  authors = book['authors']
  if TYPE_CHECKING:
    reveal_type(authors)
  authors = 'Bob'
  book['weight'] = 4.2
  del book['title']

if __name__ == "__main__":
  demo()

In [28]:
!pip install mypy

In [31]:
!mypy demo_book.py

demo_book.py:14: note: Revealed type is "builtins.list[builtins.str]"
demo_book.py:15: error: Incompatible types in assignment (expression has type "str", variable has type "list[str]")  [assignment]
demo_book.py:16: error: TypedDict "BookDict" has no key "weight"  [typeddict-unknown-key]
demo_book.py:17: error: Key "title" of TypedDict "BookDict" cannot be deleted  [misc]
Found 3 errors in 1 file (checked 1 source file)


In [35]:
# books.py
AUTHOR_ELEMENT = '<AUTHOR>{}</AUTHOR>'

def to_xml(book: BookDict) -> str: # whole point of example = using BookDict in the fcn signature
  elements: list[str] = []
  for key, value in book.items():
    if isinstance(value, list):
      elements.extend(
          AUTHOR_ELEMENT.format(n) for n in value
      )
    else:
      tag = key.upper()
      elements.append(f'<{tag}>{value}</{tag}>')
  xml = '\n\t'.join(elements)
  return f'<BOOK>\n\t{xml}\n</BOOK>'

In [36]:
!mypy books.py

Success: no issues found in 1 source file


In [37]:
!mypy books_any.py --disallow-any-expr

Success: no issues found in 1 source file


In [ ]:
# demo_not_book.py
from books import to_xml, from_json
from typing import TYPE_CHECKING

def demo() -> None:
  NOT_BOOK_JSON = """
    {"title": "Andromeda Strain",
     "flavor": "pistachio",
     "authors": true}
  """

  not_book = from_json(NOT_BOOK_JSON)
  if TYPE_CHECKING:
    reveal_type(not_book)
    reveal_type(not_book['authors'])

  print(not_book)
  print(not_book['flavor'])

  xml = to_xml(not_book)
  print(xml)

if __name__ == "__main__":
  demo()

In [38]:
!mypy demo_not_book.py

demo_not_book.py:14: note: Revealed type is "TypedDict('books.BookDict', {'isbn': builtins.str, 'title': builtins.str, 'authors': builtins.list[builtins.str], 'pagecount': builtins.int})"
demo_not_book.py:15: note: Revealed type is "builtins.list[builtins.str]"
demo_not_book.py:18: error: TypedDict "BookDict" has no key "flavor"  [typeddict-item]
Found 1 error in 1 file (checked 1 source file)


In [39]:
!python demo_not_book.py

{'title': 'Andromeda Strain', 'flavor': 'pistachio', 'authors': True}
pistachio
<BOOK>
	<TITLE>Andromeda Strain</TITLE>
	<FLAVOR>pistachio</FLAVOR>
	<AUTHORS>True</AUTHORS>
</BOOK>
